Playing ATARI-ram games using python-neat and convolutional autoencoder

# Import

In [ ]:
import numpy as np
import gym
from gym import wrappers
from __future__ import print_function
import os
%matplotlib inline
import neat
import visualize
os.environ['DISPLAY']=':0'

from six.moves import cPickle as pickle
import gym
import numpy as np
import json
from object_detection import *
from skimage.transform import resize
from skimage.color import rgb2gray
import multiprocessing
from multiprocessing.reduction import reduce_connection

# Define config

In [ ]:
fc_config_filename = 'fc.config'
game_name = 'Skiing'
game_version = 'v0'
game = game_name + '-' + game_version
num_evaluations = 1
num_cores = 11
population_size = 15

# Create environment

In [ ]:
env = gym.make(game)

# Object detection features (frame -> lowres)

In [ ]:
# fit detector and save
#odf = ObjectDetectionFeatures2(env)
odf_filename = 'odf-' + game + '.config'
#pickle.dump(odf, open(odf_filename, 'wb'))

In [ ]:
# load detector
odf = pickle.load(open(odf_filename, 'rb'))

In [ ]:
# transform for detector
output_shape = (60, 60)
#process_image = lambda x: (resize(odf.get_simple_image(x), output_shape, order=0) * 255).astype('uint8')
process_image = lambda x: (np.random.rand(60,60))

# Create neat-python population

In [ ]:
# Load configuration.
config_initial = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                             neat.DefaultSpeciesSet, neat.DefaultStagnation, fc_config_filename)

config_initial.genome_config.num_inputs = 64 #env.observation_space.shape[0]
config_initial.genome_config.num_outputs = env.action_space.n
config_initial.pop_size = population_size

In [ ]:
game_fc_config_filename = 'fc-' + game + '.config'

config_initial.save(game_fc_config_filename)

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, game_fc_config_filename)

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add reporters

In [ ]:
# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter())
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Stuff for pipes

In [ ]:
def compress_pipe(p):
    pp = pickle.dumps(reduce_connection(p))
    return(pp)
def decompress_pipe(pp):
    upw = pickle.loads(pp)
    pp = upw[0](upw[1][0],upw[1][1],upw[1][2])
    return(pp)

# Spawn cuda conv autoencoder

In [ ]:
cuda_process_p.terminate()
cuda_process_p.join()

In [54]:
m = multiprocessing.Manager()
cuda_q = m.Queue()

def cuda_process():
    print("Importing keras...")
    from keras.models import model_from_json
    from keras import backend as K
    K.set_image_dim_ordering('th')
    
    print("Loading encoder...")
    
    with open('./data/{}_Encoder_08_02.txt'.format(game_name), 'r') as model_file:
        encoder = model_from_json(json.loads(next(model_file)))

    encoder.load_weights('./data/{}_Encoder_08_02.h5'.format(game_name))
    
    print("Listening")
    global cuda_q
    while True:
        [buf, p] = cuda_q.get(block = True)
        #print("processing cuda...")
        try:
            features = encoder.predict(np.array([buf]))[0]
        except:
            features = np.zeros(64)
        p = decompress_pipe(p)
        p.send(features)
        
cuda_process_p = multiprocessing.Process(target = cuda_process, args = ())
cuda_process_p.daemon = True
cuda_process_p.start()

Importing keras...


Using Theano backend.
Using gpu device 1: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)
/home/etoestja/venv/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Loading encoder...
Listening


# Define fitness via game score

In [39]:
buf = []

def get_features(buf):
    global cuda_q
    a, b = multiprocessing.Pipe()
    b = compress_pipe(b)
    cuda_q.put([buf, b])
    features = a.recv()
    return(features)

def transform_observation(observation):
    global buf

    image = process_image(env.ale.getScreenGrayscale()[:, :, 0])
    if len(buf) < 3:
        buf.append(image)
        buf.append(image)
        buf.append(image)
    else:
        buf.pop(0)
        buf.append(image)

    features = get_features(buf)
    return features

# a = argmax_a Q(s,a)
def predict_action(observation, network):
    observation = transform_observation(observation)
    output = network.activate(observation)
    action = np.argmax(output)
    return(action)

# play num_evaluations games, take mean
def evaluate_network(env, network):
    rewards = []
    i = 0
    while i < num_evaluations:
        rewards += [get_reward(env, network)]
        i += 1

    res = np.array(rewards).mean()
    return res

# play 1 game with network
def get_reward(env, network):
    global buf
    buf = []
    observation = env.reset()
    done = False
    iteration, total_reward = 0, 0
        
    while not done:
        #env.render()
        action = predict_action(observation, network)
        observation, reward, done, info = env.step(action)
        total_reward += reward

        #if iteration % 500 == 0:
        #    print(str(iteration))

        if total_reward < -12000 or iteration >= 8000:
            break

        iteration += 1
        
    return total_reward

def evaluate_genome(genome, config):
    network = neat.nn.FeedForwardNetwork.create(genome, config)
    fitness = evaluate_network(env, network)
    return fitness

evaluator = neat.parallel.ParallelEvaluator(num_workers = num_cores, eval_function = evaluate_genome, timeout = None)

In [67]:
%timeit -n1 print(evaluate_genome(p.species.get_species(1).members[1], config))

-30000.0


KeyboardInterrupt: 

# Run evolution

In [ ]:
# Run evolution
winner = p.run(evaluator.evaluate, 200)

In [ ]:
winner = p.run(evaluator.evaluate, 2000)

# Print results

In [ ]:
# Display the winning genome.
#print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
winner_network = neat.nn.FeedForwardNetwork.create(winner, config)
visualize.draw_net(config, winner, False)
visualize.plot_stats(stats, ylog = False, view = False)
visualize.plot_species(stats, view = False)

#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
#p.run(eval_genomes, 10)

# Evaluate from checkpoint & send to OpenAI

In [ ]:
p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-1244')
p.run(evaluator.evaluate, 1)
winner = p.best_genome

In [ ]:
env_eval = gym.make(game)
monitor_path = '/tmp/' + game + '-eval'
env_eval = wrappers.Monitor(env_eval, monitor_path)
def evaluate_with_video(game, network):
    for i_episode in range(100):
        observation = env_eval.reset()
        total_reward = 0
        t = 0
        while True:
            env_eval.render()
            action = predict_action(observation, network)
            observation, reward, done, info = env_eval.step(action)
            total_reward += reward
            t += 1
            if done:
                print("Episode finished after {0} timesteps reward = {1}".format(t+1, total_reward))
                break

In [ ]:
evaluate_with_video(game, winner_network)

In [ ]:
env_eval.close()

In [ ]:
gym.upload(monitor_path, api_key='sk_ciz2F0csRzCkpESayoRuug')

# Measure performance

In [27]:
from time import time

In [56]:
# FPS total
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    features = transform_observation(1)
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

FPS: 91.4057375281


In [57]:
# FPS emulator-only
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

FPS: 178.771620251


In [58]:
# FPS OD-only
t_initial = time()
env.reset()
F = 100
buf = []
for i in range(F):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    image = process_image(env.ale.getScreenGrayscale()[:, :, 0])
t_end = time()
print("FPS: " + str(1. * F / (t_end - t_initial)))

FPS: 166.157902743


In [66]:
%timeit get_features(np.random.rand(3,60,60))

100 loops, best of 3: 5.17 ms per loop


In [63]:
buf = np.random.rand(3,60,60)
get_features(buf)

array([  27.49629211,   22.7651329 ,   63.29304504,   13.72933674,
         11.64935398,    8.80245209,   16.24185562,    8.42572975,
         13.97297478,   19.13410759,   23.83150864,   14.9833889 ,
         13.59407997,   21.5045948 ,   28.29473686,  312.12838745,
         12.93430996,   17.40763855,   20.93395615,   24.08785439,
         13.78823853,   20.31876945,    4.33094788,   10.20733356,
         17.08046913,   11.15521908,   19.00811577,   16.91891479,
         26.69672203,   12.22237682,   20.03327179,   15.56959915,
         24.43940926,   11.83578873,   16.99078751,    7.95279884,
         74.29312134,   12.99154949,   25.57496262,   19.90801239,
         23.66201591,   14.85831165,   16.7411747 ,   18.46627998,
         11.6637001 ,   15.43701839,  345.78091431,   19.35240555,
         17.32255363,   14.84072876,   25.35463905,   64.42229462,
         52.1870575 ,   21.63822746,   20.12826347,   20.74870872,
          7.12388563,    3.01123071,   20.19686508,   23.57569